# Getting data from [Information.dk](https://www.information.dk)
- Be aware of time-delay!!!

### Import packages

In [2]:
import pandas as pd
import numpy as np
import requests
import random
import time
from tqdm import tqdm
from bs4 import BeautifulSoup
import re

### Get list of dates
- Set start_dt/end_dt for the desired time span

In [5]:
from datetime import timedelta, date
def daterange(date1, date2):
    for n in range(int ((date2 - date1).days)+1):
        yield date1 + timedelta(n)
#Define the interval for possible scraping articles 
start_dt = date(2018, 8, 20)
end_dt = date(2018, 8, 20)
dates=[]
for dt in daterange(start_dt, end_dt):
    dates.append(dt.strftime("%d-%m-%Y"))

### Get list of 'Dagens Avis'  for the desired timespan

In [6]:
import requests, re, time
from bs4 import BeautifulSoup

url='https://www.information.dk/dagensavis/'
url_il=[]

for date in dates:
    url_il.append(url+date)
url_il

['https://www.information.dk/dagensavis/20-08-2018']

### Get all links within a section
- Put in the desired newspaper sections in section[]

In [7]:
sections=['indland','udland','kultur','debat','ledere','litteratur','moderne tider'] #most used sections
#be aware that one time used sections as 'omstilling' and 'højsæson' exists - i haven't found a way to search for all possible sections yet

from tqdm import tqdm
url_sub=[]

for x in tqdm(url_il):
    response=requests.get(x,headers={'User-Agent':'Mozilla/5.0'})
    if response.ok==True:
        html = response.text #raw html
        html_soup=BeautifulSoup(html,'lxml') #html parsing
        for section in sections:
            for x in html_soup.findAll('a', attrs={'href': re.compile("^/"+section)}): #finding all hyperlinks that links to subsections
                url_sub.append('https://www.information.dk'+x.get('href'))
url_sub

100%|██████████| 1/1 [00:00<00:00,  7.43it/s]


['https://www.information.dk/indland/2018/08/historien-graa-guld-paa-arbejdsmarkedet-kun-halve-sandhed-flere-aeldre-gaar-paa-foertidspension',
 'https://www.information.dk/indland/2018/08/regeringen-satser-paa-nye-uddannelser-provinsen-mens-skaerer-paa-uddannelser-provinsen',
 'https://www.information.dk/indland/2018/08/hvordan-redder-folkets-skole',
 'https://www.information.dk/indland/leder/2018/08/historien-statsminister-lykke-lars',
 'https://www.information.dk/udland/2018/08/tysklands-groenne-forener-radikal-klimapolitik-bred-appel-overhaler-afd',
 'https://www.information.dk/udland/2018/08/trumps-nye-forsvar-manafort-kan-paavirke-jury',
 'https://www.information.dk/udland/2018/08/polens-opposition-leder-magisk-formel-kan-afsaette-populisterne',
 'https://www.information.dk/udland/2018/08/kofi-annan-venlig-statsmand-begavet-diplomat',
 'https://www.information.dk/udland/2018/08/tunesiske-unge-springer-bare-mor-far',
 'https://www.information.dk/kultur/anmeldelse/2018/08/ove-k-pede

### Get information and make  DataFrame
Incl: remove all articles with 'abonnent'
- Return df with section/author, headline, subheadline, text
- It is not possible to scrape for section and author seperately. Therefore, when one should split these afterwards. 

In [8]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

articles=[]
for X in url_sub:
    time.sleep(1) #Timer: set this to avoid DOS
    html = requests.get(X).text
    soup_sub=BeautifulSoup(html,'lxml')
    if soup_sub.findAll('div', {'class':"c-node-premium-marker-title__abonnement"})==[]:
        headline=soup_sub.find('h1').getText() # Get headline
        headline_sub=soup_sub.find('div', {'class':'field field-name-field-underrubrik'}).getText() # Get sub headline  
        sec_author=soup_sub.find('div', {'class':'group-byline group-byline-desktop field-group-div'}).getText() # Get sub headline  
        main_text=soup_sub.find('div', {'class':'field field-name-body'}).getText() # Get main text
        articles.append({'Header':headline,'Header sub':headline_sub,'Section and Author':sec_author,'Text':main_text})
    else:
        continue
df=pd.DataFrame(articles) #make dataframe
df

,Header,Header sub,Section and Author,Text
0,"\n Tyskland misbruger juridiske kneb, så de...",Nu kan migranter befinde sig på tysk jord uden...,DebatSofie Møller,Tysklands indenrigsminister Horst Seehofer ha...
1,"\n Opløs de politiske partier, og giv magte...",Moderne partier kæmper for deres egen overleve...,DebatAsser Amdisen,I disse dage mødes ledelserne af de politiske...
2,\n Mennesker med handicap får udhulet deres...,Udhulingen af overførselsindkomster bør stoppe...,DebatThorkild Olesen,Dansk Folkeparti vil stoppe udhulingen af fol...
3,\n Bøfafgift vil sende produktion ud af lan...,Vi bør ikke indføre kødafgifter. De vil formen...,DebatErling Bonnesen,I Danmark skal vi ikke pålægge afgifter på kø...
4,\n København svigter de udsatte\n,"Hvis København skal være en by for alle, skal ...",DebatMia Nyegaard,"Sommeren har indtil videre været fantastisk, ..."


### Furhter work
- Scrape comments in each article
- Clean the information (e.g. the \n in title and divide author and section in one)
- Optimize code? We want to write our loops in function, because it will make errors more intuitive (from lecture)

### Other data possible to get
- Name of Photographer
- Education of author
- Amount of shares (Can be used as an approximation of how viral the article went)




In [13]:
df['Text'][1]

' I disse dage mødes ledelserne af de politiske partier til sommergruppemøder.\nEt tema bliver uden tvivl, hvordan partierne skal\xa0forholde sig til fremtidens demokrati, men i virkeligheden er det helt forfejlet. Partierne kan ikke finde løsningen på demokratiets problemer – de er nemlig selv (et af) demokratiets problemer.\nInden demokratiet blev indført, var det helt accepteret, at bestemte grupper overtog magten i et samfund. Det kunne være jordejere eller krigere, som kaldte sig en adel og derfor mente, at den havde ret til at bestemme over resten af landets indbyggere.\nDet kunne være en religiøs kaste, som mente, at deres nærhed til Gud gav dem særlige rettigheder. Fælles for dem var, at medlemskab af grupperne var nok til at få indflydelse.\nEfter demokratiets indførelse var det et mål for det nye demokrati at afskaffe sådanne privilegerede kaster, men da der ret hurtigt opstod et nyt partivælde, blev systemet genindført af bagvejen.\nUsund forretning\nModerne partier er organ

In [14]:
count(df['Text'][1])

NameError: name 'count' is not defined

In [15]:
from collections import Counter
counts = Counter(df['Text'][1])
print(counts)

Counter({' ': 685, 'e': 668, 'r': 382, 't': 286, 'n': 285, 'i': 263, 'd': 227, 's': 208, 'a': 206, 'o': 182, 'l': 172, 'g': 147, 'm': 146, 'k': 135, 'v': 88, 'f': 87, 'p': 82, ',': 63, 'b': 62, 'u': 51, '\n': 45, 'æ': 37, 'h': 35, '.': 34, 'å': 29, 'ø': 28, 'y': 16, 'D': 11, 'j': 10, '\xa0': 8, 'c': 8, 'P': 7, 'E': 6, 'I': 4, '–': 4, 'G': 3, 'F': 3, 'H': 3, 'L': 3, 'A': 3, '(': 2, ')': 2, 'M': 2, 'V': 2, 'N': 2, 'S': 2, '’': 2, 'U': 1, 'K': 1, '!': 1, '2': 1, '5': 1, '-': 1, '*': 1})


SyntaxError: invalid syntax (<ipython-input-18-34c1b380b03e>, line 7)

In [ ]:
https://sparenergi.dk/offentlig/vaerktoejer/energi-og-co2-regnskab/aarhus?year=2015